# Convert bounding boxes to instance segmentation masks using SAM

![img](../images/bb2seg.jpg)

This notebook creates a derived Table with an added column containing instance segmentation masks generated by the [SAM model](https://github.com/facebookresearch/segment-anything), using the input Table's bounding boxes as prompts.

## Install dependencies

In [ ]:
%%capture

%pip install 3lc
%pip install git+https://github.com/3lc-ai/3lc-examples.git

## Imports

In [ ]:
import tlc

from tlc_tools.sam_autosegment import bbs_to_segments

## Project Setup

In [ ]:
PROJECT_NAME = "3LC Tutorials - COCO128"
MODEL_TYPE = "vit_b"
CHECKPOINT = "../../transient_data/sam_vit_b_01ec64.pth"

## Load the input table

Load the input table, created in [create-table-from-coco.ipynb](../1-create-tables/create-table-from-coco.ipynb).

In [ ]:
input_table = tlc.Table.from_names("initial", "COCO128", PROJECT_NAME)

## Run SAM model

In [ ]:
out_table = bbs_to_segments(
    input_table,
    sam_model_type=MODEL_TYPE,
    checkpoint=CHECKPOINT,
    description="Added segmentation column from bounding boxes",
)

## Visualize an example mask

In [ ]:
example_mask = out_table[3]["segments"]["masks"]
print(example_mask.shape)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
plt.imshow(example_mask[:, :, 0], cmap="gray")
plt.axis("off")
plt.show()